In [1]:
import random
import math
from collections import deque
import pandas as pd
import datetime

In [7]:
def load_words_from_csv(file_path=None):
    if file_path:
        try:
            df = pd.read_csv(file_path)
            if 'English' not in df.columns or 'Vietnamese' not in df.columns:
                raise ValueError("CSV must have 'English' and 'Vietnamese' columns")
            word_list = [(row['English'].strip(), row['Vietnamese'].strip()) for _, row in df.iterrows()]
        except Exception as e:
            print(f"Error reading CSV: {e}")
            print("Using default word list")
            word_list = []
    else:
        # Default word list
        word_pairs_raw = [
            "apple - quả táo",
            "dog - con chó",
            "book - quyển sách",
            "car - xe hơi",
            "water - nước",
            "computer - máy tính",
            "house - ngôi nhà",
            "school - trường học",
            "pen - cây bút",
            "table - cái bàn"
        ]
        word_list = [(en.strip(), vi.strip()) for item in word_pairs_raw for en, vi in [item.split(" - ")]]
    
    # Initialize Spaced Repetition parameters
    word_data = [
        {
            'en': en,
            'vi': vi,
            'interval': 1,  # Initial interval (days)
            'ease': 2.5,   # Initial ease factor
            'repetitions': 0,  # Number of successful reviews
            'next_review': datetime.datetime.now()  # Immediate review
        }
        for en, vi in word_list
    ]
    return word_data

# Example usage: Replace with your CSV file path or set to None for default
word_data = load_words_from_csv("C:/Users/ADMIN/Documents/PROJECT/Enku/input.csv")  # Use None for default or provide "path/to/your/file.csv"
random.shuffle(word_data)

Creat question

In [8]:
def create_question(word_item, all_meanings):
    correct_meaning = word_item['vi']
    # Select 7 random wrong meanings
    wrong_meanings = random.sample([m for m in all_meanings if m != correct_meaning], 7)
    options = wrong_meanings + [correct_meaning]
    random.shuffle(options)
    return word_item['en'], options, correct_meaning

Space repetition

In [9]:
def update_spaced_repetition(word_item, quality):
    """
    Update word's Spaced Repetition parameters based on user performance.
    Quality: 0 (wrong), 3 (correct with effort), 5 (correct easily)
    """
    now = datetime.datetime.now()
    
    if quality >= 3:  # Correct answer
        word_item['repetitions'] += 1
        if word_item['repetitions'] == 1:
            word_item['interval'] = 1
        elif word_item['repetitions'] == 2:
            word_item['interval'] = 6
        else:
            word_item['interval'] = math.ceil(word_item['interval'] * word_item['ease'])
        
        # Update ease factor
        word_item['ease'] = max(1.3, word_item['ease'] + 0.1 - (5 - quality) * (0.08 + (5 - quality) * 0.02))
        
        # Schedule next review
        word_item['next_review'] = now + datetime.timedelta(days=word_item['interval'])
    else:  # Wrong answer
        word_item['repetitions'] = 0
        word_item['interval'] = 1
        word_item['ease'] = max(1.3, word_item['ease'] - 0.2)
        word_item['next_review'] = now  # Review immediately

    return word_item

In [10]:
queue = deque(word_data)
correct_count = 0
total = len(word_data)
stop_flag = False

def check_stop(value):
    global stop_flag
    if value.strip().upper() == "STOP":
        stop_flag = True

now = datetime.datetime.now()
while queue and not stop_flag:
    # Get the next word that's due for review
    word_item = queue.popleft()
    if word_item['next_review'] > now:
        queue.append(word_item)  # Not due yet, put back
        continue
    
    word, options, correct_meaning = create_question(word_item, [item['vi'] for item in word_data])
    
    print(f"\nTừ: {word}")
    for idx, opt in enumerate(options):
        print(f"{idx + 1}. {opt}")
    
    user_input = input("Chọn đáp án (1-8) hoặc gõ 'STOP' để dừng: ").strip()
    check_stop(user_input)
    if stop_flag:
        break
    
    try:
        answer = int(user_input)
        if not 1 <= answer <= len(options):
            raise ValueError
        chosen = options[answer - 1]
    except:
        print("❌ Nhập không hợp lệ hoặc sai cú pháp! Tự động tính sai.")
        chosen = None
    
    quality = 0  # Default: wrong
    if chosen == correct_meaning:
        print("✅ Đúng!")
        correct_count += 1
        quality = 5  # Correct easily
    else:
        print(f"❌ Sai! Đáp án đúng là: {correct_meaning}")
        while True:
            example = input("Nhập ví dụ với từ này (bắt buộc) hoặc 'STOP' để dừng: ").strip()
            check_stop(example)
            if stop_flag:
                break
            if example:
                break
            else:
                print("⚠️ Không được bỏ trống!")
        if stop_flag:
            break
    
    # Update Spaced Repetition parameters
    word_item = update_spaced_repetition(word_item, quality)
    queue.append(word_item)

print(f"\n🎉 Kết thúc luyện tập! Đúng {correct_count}/{total}.")


Từ: sun
1. mặt trời
2. quển sách
3. con chó
4. trường học
5. xe hơi
6. hoa
7. máy tính
8. cây bút
❌ Sai! Đáp án đúng là: mặt trời

Từ: car
1. trường học
2. cây bút
3. quển sách
4. máy tính
5. quả táo
6. hoa
7. chim
8. xe hơi
✅ Đúng!

Từ: water
1. quả táo
2. mặt trăng
3. con chó
4. mặt trời
5. ngôi nhà
6. máy tính
7. nước
8. chim
✅ Đúng!

Từ: moon
1. hoa
2. cây bút
3. máy tính
4. mặt trăng
5. cái bàn
6. mặt trời
7. trường học
8. chim
❌ Sai! Đáp án đúng là: mặt trăng

🎉 Kết thúc luyện tập! Đúng 2/15.
